In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# openai-community/gpt2-xl
model = AutoModelForCausalLM.from_pretrained("openai-community/gpt2-xl")
tokenizer = AutoTokenizer.from_pretrained("openai-community/gpt2-xl")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/689 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/6.43G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

In [ ]:
input = "I go to work as an AI Engineer"
tokens = tokenizer(input, return_tensors="pt").to(model.device)

output = model.generate(**tokens, max_new_tokens=50, do_sample=True, pad_token_id=tokenizer.eos_token_id)

print(tokenizer.decode(output[0], skip_special_tokens=True))

I go to work as an AI Engineer and am very much aware that people in my line of work are often compared to machines. It is true that we do have to consider our ability to react in different circumstances while at the same time we are also told we are working towards artificial intelligence.


# **Train GPT from scratch**

In [ ]:
# Load dataset
from datasets import load_dataset

ds = load_dataset("datablations/c4-filter-small", split="train")
ds = ds.select_columns(["text"])
ds = ds.train_test_split(test_size=0.1)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/791 [00:00<?, ?B/s]

data/train-00000-of-00001-091e566583af27(…):   0%|          | 0.00/141M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/100000 [00:00<?, ? examples/s]

In [ ]:
# Tạo bộ tokenizer cho GPT
from tokenizers import Tokenizer
from tokenizers.models import BPE  # Byte-Pair Encoding
from tokenizers.trainers import BpeTrainer
from tokenizers.pre_tokenizers import ByteLevel
from tokenizers.normalizers import NFKC
from tokenizers.decoders import ByteLevel as ByteLevelDecoder

In [ ]:
# GPT-2 sử dụng Byte-Pair Encoding làm tokenizer
tokenizer = Tokenizer(BPE())
# Sử dụng pre-tokenizer để xử lý Unicode và khoảng trắng
tokenizer.pre_tokenizer = ByteLevel()
# Chuẩn hoá văn bản (NKFC: Normalization Form Compability Composition)
tokenizer.normalizer = NFKC()
# Decoder byte-level để decode token về text
tokenizer.decoder = ByteLevelDecoder()

# Thiết lập bộ Trainer cho BPE với vocab size là 50257 (giống GPT-2)
# Thêm speical token: (BOS, PAD, EOS, UNK, MASK)
trainer = BpeTrainer(
    vocab_size=50257,
    speical_token=["<s>", "<pad>", "</s>", "<unk>", "<mask>"]
)

tokenizer.train_from_iterator(ds["train"]["text"], trainer)
tokenizer.save("gpt_tokenizer.json")

In [ ]:
# Tạo bộ tokenizer cho kiến trúc Transformer mà GPT-2 sử dụng
from transformers import PreTrainedTokenizerFast

# Transformers yêu cầu tokenizer ở định dạng PreTrainedTokenizerFast
# Nếu thiếu => GPT-2 sẽ gặp lỗi trong quá trình token hoá text để generate
tokenizer = PreTrainedTokenizerFast(tokenizer_file="gpt_tokenizer.json")
tokenizer.add_special_tokens({
    "bos_token": "<s>",
    "eos_token": "</s>",
    "unk_token": "<unk>",
    "pad_token": "<pad>",
    "mask_token": "<mask>"
})

tokenizer.save_pretrained("gpt-tokenizer")

('gpt-tokenizer/tokenizer_config.json',
 'gpt-tokenizer/special_tokens_map.json',
 'gpt-tokenizer/tokenizer.json')

In [ ]:
def tokenize(text):
    return tokenizer(text["text"])

tokenized_ds = ds.map(
    tokenize, remove_columns=["text"], batched=True, num_proc=20
)

Map (num_proc=20):   0%|          | 0/90000 [00:00<?, ? examples/s]

Map (num_proc=20):   0%|          | 0/10000 [00:00<?, ? examples/s]

In [ ]:
block_size = 512
# block_size càng lớn => ngữ cảnh nhiều hơn => học hiểu ngữ cảnh tốt hơn
# GPT thường dùng block_size từ 512 đến 1024

def group_texts(sentences):
    """
    sentences là một dict từ batch: input_ids, token_type_ids, attention_mask
    mỗi value là list của lists
    """
    # concat input_ids
    # Nối tất cả các id trong các list thành một list dài duy nhất
    # ví dụ: có 3 list trong list [[1, 2], [3, 4, 5], [6]] => [1, 2, 3, 4, 5, 6]
    concatenated = {k: sum(sentences[k], []) for k in sentences.keys()}
    total_length = len(concatenated["input_ids"])
    total_length = (total_length // block_size) * block_size

    # Chia các block size
    # concatenated["input_ids"] = [1, 2, 3, 4, ...., 1024] => [[1, ..., 512], [513, ...,1024]]
    result = {
        k: [concatenated[k][i : i + block_size] for i in range(0, total_length, block_size)] # Từ i=0 đến total_length, step=block_size, slice[i:i+512]
        for k in concatenated
    }

    result["labels"] = result["input_ids"].copy()
    return result

In [ ]:
lm_ds = tokenized_ds.map(group_texts, batched=True, num_proc=20)

Map (num_proc=20):   0%|          | 0/90000 [00:00<?, ? examples/s]

Map (num_proc=20):   0%|          | 0/10000 [00:00<?, ? examples/s]

In [ ]:
# Xây dựng kiến trúc của GPT thông qua thư viện GPT2Config và GPT2LMHeadModel
from transformers import GPT2Config, GPT2LMHeadModel

config = GPT2Config(
    vocab_size = tokenizer.vocab_size,
    n_position=512,
    n_ctx=512,
    n_embed=512,
    n_layer=6, # 6 kiến trúc Transformer cho GPT-2
    n_head=8,
    bos_token_id = tokenizer.bos_token_id,
    eos_token_id = tokenizer.eos_token_id
)

model = GPT2LMHeadModel(config)

In [ ]:
from transformers import Trainer, TrainingArguments, DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=False
)

training_args = TrainingArguments(
    output_dir="gpt-small-c4",
    logging_dir="logs",
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=20,
    eval_strategy="steps",
    save_strategy="steps",
    logging_strategy="steps",
    eval_steps=1000,
    save_steps=1000,
    logging_steps=1000,
    save_total_limit=1,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    load_best_model_at_end=True,
    fp16=True
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_ds["train"],
    eval_dataset=lm_ds["test"],
    processing_class=tokenizer,
    data_collator=data_collator
)

In [ ]:
trainer.train()

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'pad_token_id': 50260}.
/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: cybersoft-codingcamp (cybersoft-codingcamp-cybersoft-academy-o-t-o-chuy-n-gia-) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


`loss_type=None` was set in the config but it is unrecognized. Using the default loss: `ForCausalLMLoss`.


Step,Training Loss,Validation Loss
